# Tutorial / ejemplo de (\_\_str\_\_ y todos esos)

## What Are Dunder Methods?
In Python, special methods are a set of predefined methods you can use to enrich your classes. They are easy to recognize because they start and end with double underscores, for example \_\_init\_\_ or \_\_str\_\_.

As it quickly became tiresome to say under-under-method-under-under Pythonistas adopted the term “dunder methods”, a short form of “double under.”

These “dunders” or “special methods” in Python are also sometimes called “magic methods.” But using this terminology can make them seem more complicated than they really are—at the end of the day there’s nothing “magical” about them. You should treat these methods like a normal language feature.

Dunder methods let you emulate the behavior of built-in types. For example, to get the length of a string you can call len('string'). But an empty class definition doesn’t support this behavior out of the box:

In [1]:
class NoLenSupport:
    pass

obj = NoLenSupport()
len(obj)

TypeError: object of type 'NoLenSupport' has no len()

To fix this, you can add a \_\_len\_\_ dunder method to your class:

In [2]:
class LenSupport:
    def __len__(self):
        return 42

obj = LenSupport()
len(obj)

42

Another example is slicing. You can implement a \_\_getitem\_\_ method which allows you to use Python’s list slicing syntax: obj\[ start : stop \].

## Special Methods and the Python Data Model

This elegant design is known as the [Python data model](https://docs.python.org/3/reference/datamodel.html) and lets developers tap into rich language features like sequences, iteration, operator overloading, attribute access, etc.

You can see Python’s data model as a powerful API you can interface with by implementing one or more dunder methods. If you want to write more Pythonic code, knowing how and when to use dunder methods is an important step.

For a beginner this might be slightly overwhelming at first though. No worries, in this article I will guide you through the use of dunder methods using a simple Account class as an example.

## Enriching a Simple Account Class

Throughout this article I will enrich a simple Python class with various dunder methods to unlock the following language features:

- Initialization of new objects
- Object representation
- Enable iteration
- Operator overloading (comparison)
- Operator overloading (addition)
- Method invocation
- Context manager support (with statement)

## Enriching Your Python Classes with Dunder (Special) Methods

### Construction

Right upon starting my class I already need a special method. To construct account objects from the Account class I need a constructor which in Python is the \_\_init\_\_ dunder:

In [4]:
class Account:
    'A simple account class'
    
    def __init__(self, owner, amount=0):
        'This is the constructor that lets us create objects from this class'
        self.owner = owner
        self.amount = amount
        self._transactions = []

The constructor takes care of setting up the object. In this case it receives the owner name, an optional start amount and defines an internal transactions list to keep track of deposits and withdrawals.

This allows us to create new accounts like this:

In [5]:
acc1 = Account('bob')
acc1

In [6]:
acc2 = Account('bob', 10)
acc2

### Object Representation: \_\_str\_\_, \_\_repr\_\_

It’s common practice in Python to provide a string representation of your object for the consumer of your class (a bit like API documentation.) There are two ways to do this using dunder methods:

\_\_repr\_\_: The “official” string representation of an object. This is how you would make an object of the class. The goal of \_\_repr\_\_ is to be unambiguous.

\_\_str\_\_: The “informal” or nicely printable string representation of an object. This is for the enduser.

Let’s implement these two methods on the Account class:


In [7]:
class Account:
    'A simple account class'
    
    def __init__(self, owner, amount=0):
        'This is the constructor that lets us create objects from this class'
        self.owner = owner
        self.amount = amount
        self._transactions = []
    
    def __repr__(self):
        return 'Account({!r}, {!r})'.format(self.owner, self.amount)
    
    def __str__(self):
        return 'Account of {} with starting amount: {}'.format(self.owner, self.amount)

In [8]:
acc1 = Account('bob')
acc2 = Account('bob', 10)

If you don’t want to hardcode "Account" as the name for the class you can also use self.\_\_class\_\_.\_\_name\_\_ to access it programmatically.

If you wanted to implement just one of these to-string methods on a Python class, make sure it’s \_\_repr\_\_. (cuando python quiere usar \_\_str\_\_ y no la encuentra, el comportamiento por default es usar \_\_repr\_\_, entonces con definir esta ultima me aseguro de que tengo una descripcion razonable siempre)

Now I can query the object in various ways and always get a nice string representation:

In [9]:
repr(acc1)

"Account('bob', 0)"

In [10]:
str(acc1)

'Account of bob with starting amount: 0'

In [11]:
print(acc2)

Account of bob with starting amount: 10


In [12]:
lista = [acc1, acc1] # al incluir objetos en una lista usa __repr__ para mostrar una descripcion
lista

[Account('bob', 0), Account('bob', 0)]

In [13]:
type(lista[0]) # pero obviamente sigue siendo el objeto (no lo convierte a string)

__main__.Account

In [14]:
lista[0].owner

'bob'

In [15]:
print(lista) # aunque imprima la lista no usa str, porque estan dentro de la lista

[Account('bob', 0), Account('bob', 0)]


In [16]:
print(lista[0]) # ahí sí usa __str__

Account of bob with starting amount: 0


In [17]:
print(f'cuenta 1 con str: {acc1}  ---  cuenta 2 con repr: {acc2!r}') # en acc2 fuerzo el uso de __repr__ en un print

cuenta 1 con str: Account of bob with starting amount: 0  ---  cuenta 2 con repr: Account('bob', 10)


### Iteration: \_\_len\_\_, \_\_getitem\_\_, \_\_reversed\_\_

In order to iterate over our account object I need to add some transactions. So first, I’ll define a simple method to add transactions. I’ll keep it simple because this is just setup code to explain dunder methods, and not a production-ready accounting system.

- Se agrega el metodo add_transaction que agrega transacciones a una lista
- Se agrego el metodo balance que calcula el saldo sumando los movimientos al monto de apertura. 
- A este ultimo metodo se le agregó el decorator @property para poderlo acceder directamente como account.balance

1ro ver que la claze como estaba definida hasta ahora no acepta len() y tampoco se puede iterar...


In [18]:
len(acc1)

TypeError: object of type 'Account' has no len()

In [19]:
for t in acc1:
    print(t)

TypeError: 'Account' object is not iterable

In [20]:
acc1[1]

TypeError: 'Account' object is not subscriptable

In [21]:
class Account:
    'A simple account class'
    
    def __init__(self, owner, amount=0):
        'This is the constructor that lets us create objects from this class'
        self.owner = owner
        self.amount = amount
        self._transactions = []
    
    def __repr__(self):
        return 'Account({!r}, {!r})'.format(self.owner, self.amount)
    
    def __str__(self):
        return 'Account of {} with starting amount: {}'.format(self.owner, self.amount)
    
    def add_transaction(self, amount):
        if not isinstance(amount, int):
            raise ValueError('please use int for amount')
        self._transactions.append(amount)
        
    @property
    def balance(self):
        return self.amount + sum(self._transactions)
    
    def __len__(self):
        return len(self._transactions)
    
    def __getitem__(self, position):
        return self._transactions[position]

    # updated to reverse the normal iteration order (https://dbader.org/blog/python-dunder-methods - comment Pablo Ziliani)
    def __reversed__(self):
        return self[::-1]

In [22]:
acc1 = Account('bob')
acc1.add_transaction(20)
acc1.add_transaction(-10)
acc1.add_transaction(50)
acc1.add_transaction(-20)
acc1.add_transaction(30)

acc1.balance
80

80

In [23]:
len(acc1)

5

In [24]:
for t in acc1:
    print(t)

20
-10
50
-20
30


In [25]:
acc1[0]

20

In [26]:
acc1[-2:]

[-20, 30]

In [27]:
sorted(acc1)

[-20, -10, 20, 30, 50]

In [28]:
reversed(acc1)

[30, -20, 50, -10, 20]

### Operator Overloading for Comparing Accounts: \_\_eq\_\_, \_\_lt\_\_

We all write dozens of statements daily to compare Python objects:

\>\>\> 2 > 1  
True

\>\>\> 'a' > 'b'  
False
    
This feels completely natural, but it’s actually quite amazing what happens behind the scenes here. Why does > work equally well on integers, strings and other objects (as long as they are the same type)? This polymorphic behavior is possible because these objects implement one or more comparison dunder methods.

An easy way to verify this is to use the dir() builtin:  

Notar los metodos \_\_eq\_\_, \_\_ge\_\_ y \_\_gt\_\_


In [29]:
dir('a')

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'capitalize',
 'casefold',
 'center',
 'count',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isascii',
 'isdecimal',
 'isdigit',
 'isidentifier',
 'islower',
 'isnumeric',
 'isprintable',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'ljust',
 'lower',
 'lstrip',
 'maketrans',
 'partition',
 'replace',
 'rfind',
 'rindex',
 'rjust',
 'rpartition',
 'rsplit',
 'rstrip',
 'split',
 'splitlines',
 'startswith',
 'strip',
 'swapcase',
 'title',
 'translate',
 'upper',


Let’s build a second account object and compare it to the first one (I am adding a couple of transactions for later use):

In [30]:
acc2 = Account('tim', 100)

In [31]:
acc2.add_transaction(20)
acc2.add_transaction(40)
acc2.balance

160

In [32]:
acc2 > acc1

TypeError: '>' not supported between instances of 'Account' and 'Account'

In [33]:
from functools import total_ordering # decorator para no tener que implementar todas y cada una de las comparaciones (solo eq y lt)

@total_ordering
class Account:
    'A simple account class'
    
    def __init__(self, owner, amount=0):
        'This is the constructor that lets us create objects from this class'
        self.owner = owner
        self.amount = amount
        self._transactions = []
    
    def __repr__(self):
        return 'Account({!r}, {!r})'.format(self.owner, self.amount)
    
    def __str__(self):
        return 'Account of {} with starting amount: {}'.format(self.owner, self.amount)
    
    def add_transaction(self, amount):
        if not isinstance(amount, int):
            raise ValueError('please use int for amount')
        self._transactions.append(amount)
        
    @property
    def balance(self):
        return self.amount + sum(self._transactions)
    
    def __len__(self):
        return len(self._transactions)

    def __reversed__(self):
        return self[::-1]
    
    def __getitem__(self, position):
        return self._transactions[position]

    # Con implementar eq y lt alcanza para que funcionen todas las posibles comparaciones (==, >, <, >=, <=)
    def __eq__(self, other):
        return self.balance == other.balance
        
    def __lt__(self, other):
        return self.balance < other.balance

y ahora si...

In [34]:
acc1 = Account('bob')
acc2 = Account('tim', 100)
print(acc1.balance)
print(acc2.balance)

0
100


In [35]:
acc1 == acc2

False

In [36]:
acc1 > acc2

False

In [37]:
acc2 < acc1

False

In [38]:
acc1.add_transaction(110)
print(acc1.balance)
print(acc2.balance)

110
100


In [39]:
acc1 >= acc2

True

In [40]:
acc2.add_transaction(10)
print(acc1.balance)
print(acc2.balance)
acc1 != acc2

110
110


False

### Operator Overloading for Merging Accounts: \_\_add\_\_

In Python, everything is an object. We are completely fine adding two integers or two strings with the + (plus) operator, it behaves in expected ways:

\>\>\> 1 + 2  
3  

\>\>\> 'hello' + ' world'  
'hello world'  

Again, we see polymorphism at play: Did you notice how + behaves different depending the type of the object? For integers it sums, for strings it concatenates. Again doing a quick dir() on the object reveals the corresponding “dunder” interface into the data model:

\>\>\> dir(1)  
\[...  
'\_\_add\_\_',  
...  
'\_\_radd\_\_',  
...\]  

Our Account object does not support addition yet, so when you try to add two instances of it there’s a TypeError:


In [41]:
acc3 = Account('james', 200)

In [42]:
acc1 + acc3

TypeError: unsupported operand type(s) for +: 'Account' and 'Account'

Let’s implement \_\_add\_\_ to be able to merge two accounts. The expected behavior would be to merge all attributes together: the owner name, as well as starting amounts and transactions. To do this we can benefit from the iteration support we implemented earlier:


In [43]:
from functools import total_ordering

@total_ordering
class Account:
    'A simple account class'
    
    def __init__(self, owner, amount=0):
        'This is the constructor that lets us create objects from this class'
        self.owner = owner
        self.amount = amount
        self._transactions = []
    
    def __repr__(self):
        return 'Account({!r}, {!r})'.format(self.owner, self.amount)
    
    def __str__(self):
        return 'Account of {} with starting amount: {}'.format(self.owner, self.amount)
    
    def add_transaction(self, amount):
        if not isinstance(amount, int):
            raise ValueError('please use int for amount')
        self._transactions.append(amount)
        
    @property
    def balance(self):
        return self.amount + sum(self._transactions)
    
    def __len__(self):
        return len(self._transactions)
    
    def __getitem__(self, position):
        return self._transactions[position]

    def __reversed__(self):
        return self[::-1]
    
    def __eq__(self, other):
        return self.balance == other.balance
        
    def __lt__(self, other):
        return self.balance < other.balance

    def __add__(self, other):
        owner = '{} & {}'.format(self.owner, other.owner)
        start_amount = self.amount + other.amount
        acc = Account(owner, start_amount)
        for t in list(self) + list(other):
            acc.add_transaction(t)
        return acc
    
    # En algunos casos, ej: si uso la funcion sum sobre una lista de objetos de esta clase, por la forma en que esta
    # implementada (inicializar la suma en 0 y sumar uno a uno los elementos), trataria de ejecutar 0.__add__(acc1) al no estar
    # definida para sumar un entero a un objeto de este tipo, tratara de ejecutar acc1.__radd__(0).
    # Implemento este metodo para que al calcular sum[acc1, acc2, acc3, ...] se logre 'sumar' los objetos de la lista en el sentido
    # que le da __add__()
    def __radd__(self, other):
        if other == 0:
            return self
        else:
            return self.__add__(other)

In [44]:
acc1 = Account('bob', 0)
acc1.add_transaction(10)
acc1.add_transaction(-20)
acc1.add_transaction(30)
acc2 = Account('tim', 100)
acc2.add_transaction(12)
acc2.add_transaction(-42)
acc3 = Account('james', 200)
acc3.add_transaction(44)
acc3.add_transaction(34)
acc3.add_transaction(64)
acc3.add_transaction(-14)

In [45]:
acc1 + acc2

Account('bob & tim', 100)

In [46]:
jointAccount = acc2 + acc3
print(jointAccount)
for t in jointAccount:
    print(t)
print('balance', jointAccount.balance)

Account of tim & james with starting amount: 300
12
-42
44
34
64
-14
balance 398


In [47]:
todas = sum([acc1, acc2, acc3]) ### Esto funca gracias a haber definido __radd__
print(todas)
print('transacciones',list(todas),'total:',sum(list(todas)))
print('balance',todas.balance)


Account of bob & tim & james with starting amount: 300
transacciones [10, -20, 30, 12, -42, 44, 34, 64, -14] total: 118
balance 418


Yes, it is a bit more involved than the other dunder implementations so far. It should show you though that you are in the driver’s seat. You can implement addition however you please. If we wanted to ignore historic transactions—fine, you can also implement it like this:

<code>
def __add__(self, other):  
    owner = self.owner + other.owner  
    start_amount = self.balance + other.balance  
    return Account(owner, start_amount)  
<\code>

### Method invocation: \_\_call\_\_

You can make an object callable like a regular function by adding the \_\_call\_\_ dunder method. For our account class we could print a nice report of all the transactions that make up its balance:


In [48]:
from functools import total_ordering

@total_ordering
class Account:
    'A simple account class'
    
    def __init__(self, owner, amount=0):
        'This is the constructor that lets us create objects from this class'
        self.owner = owner
        self.amount = amount
        self._transactions = []
    
    def __repr__(self):
        return 'Account({!r}, {!r})'.format(self.owner, self.amount)
    
    def __str__(self):
        return 'Account of {} with starting amount: {}'.format(self.owner, self.amount)
    
    def add_transaction(self, amount):
        if not isinstance(amount, int):
            raise ValueError('please use int for amount')
        self._transactions.append(amount)
        
    @property
    def balance(self):
        return self.amount + sum(self._transactions)
    
    def __len__(self):
        return len(self._transactions)
    
    def __getitem__(self, position):
        return self._transactions[position]

    def __reversed__(self):
        return self[::-1]
    
    def __eq__(self, other):
        return self.balance == other.balance
        
    def __lt__(self, other):
        return self.balance < other.balance

    def __add__(self, other):
        owner = '{}&{}'.format(self.owner, other.owner)
        start_amount = self.amount + other.amount
        acc = Account(owner, start_amount)
        for t in list(self) + list(other):
            acc.add_transaction(t)
        return acc

    def __call__(self):
        print('Start amount: {}'.format(self.amount))
        print('Transactions: ')
        for transaction in self:
            print(transaction)
        print('\nBalance: {}'.format(self.balance))

Now when I call the object with the double-parentheses acc() syntax, I get a nice account statement with an overview of all transactions and the current balance:

In [49]:
acc1 = Account('bob', 10)
acc1.add_transaction(20)
acc1.add_transaction(-10)
acc1.add_transaction(50)
acc1.add_transaction(-20)
acc1.add_transaction(30)

acc1()

Start amount: 10
Transactions: 
20
-10
50
-20
30

Balance: 80


Please keep in mind that this is just a toy example. A “real” account class probably wouldn’t print to the console when you use the function call syntax on one of its instances. In general, the downside of having a \_\_call\_\_ method on your objects is that it can be hard to see what the purpose of calling the object is.

Most of the time it’s therefore better to add an explicit method to the class. In this case it probably would’ve been more transparent to have a separate Account.print_statement() method.

### Context Manager Support and the With Statement: \_\_enter\_\_, \_\_exit\_\_

My final example in this tutorial is about a slightly more advanced concept in Python: Context managers and adding support for the with statement.

Now, what is a “context manager” in Python? Here’s a quick overview:

A context manager is a simple “protocol” (or interface) that your object needs to follow so it can be used with the with statement. Basically all you need to do is add \_\_enter\_\_ and \_\_exit\_\_ methods to an object if you want it to function as a context manager.

Let’s use context manager support to add a rollback mechanism to our Account class. If the balance goes negative upon adding another transaction we rollback to the previous state.

We can leverage the Pythonic with statement by adding two more dunder methods. I’m also adding some print calls to make the example clearer when we demo it:


In [50]:
from functools import total_ordering

@total_ordering
class Account:
    'A simple account class'
    
    def __init__(self, owner, amount=0):
        'This is the constructor that lets us create objects from this class'
        self.owner = owner
        self.amount = amount
        self._transactions = []
    
    def __repr__(self):
        return 'Account({!r}, {!r})'.format(self.owner, self.amount)
    
    def __str__(self):
        return 'Account of {} with starting amount: {}'.format(self.owner, self.amount)
    
    def add_transaction(self, amount):
        if not isinstance(amount, int):
            raise ValueError('please use int for amount')
        self._transactions.append(amount)
        
    @property
    def balance(self):
        return self.amount + sum(self._transactions)
    
    def __len__(self):
        return len(self._transactions)
    
    def __getitem__(self, position):
        return self._transactions[position]

    def __reversed__(self):
        return self[::-1]
    
    def __eq__(self, other):
        return self.balance == other.balance
        
    def __lt__(self, other):
        return self.balance < other.balance

    def __add__(self, other):
        owner = '{}&{}'.format(self.owner, other.owner)
        start_amount = self.amount + other.amount
        acc = Account(owner, start_amount)
        for t in list(self) + list(other):
            acc.add_transaction(t)
        return acc

    def __call__(self):
        print('Start amount: {}'.format(self.amount))
        print('Transactions: ')
        for transaction in self:
            print(transaction)
        print('\nBalance: {}'.format(self.balance))
        
    def __enter__(self):
        print('ENTER WITH: making backup of transactions for rollback')
        self._copy_transactions = list(self._transactions)
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        print('EXIT WITH:', end=' ')
        if exc_type:
            self._transactions = self._copy_transactions
            print('rolling back to previous transactions')
            print('transaction resulted in {} ({})'.format(exc_type.__name__, exc_val))
        else:
            print('transaction ok')

As an exception has to be raised to trigger a rollback, I define a quick helper method to validate the transactions in an account:

In [51]:
def validate_transaction(acc, amount_to_add):
    with acc as a:
        print('adding {} to account'.format(amount_to_add))
        a.add_transaction(amount_to_add)
        print('new balance would be: {}'.format(a.balance))
        if a.balance < 0:
            raise ValueError('sorry cannot go in debt!')


In [52]:
acc4 = Account('sue', 10)

print('\nBalance start: {}'.format(acc4.balance))
validate_transaction(acc4, 20)

print('\nBalance end: {}'.format(acc4.balance))


Balance start: 10
ENTER WITH: making backup of transactions for rollback
adding 20 to account
new balance would be: 30
EXIT WITH: transaction ok

Balance end: 30


In [53]:
acc4 = Account('sue', 10)

print('\nBalance start: {}'.format(acc4.balance))
try:
    validate_transaction(acc4, -50)
except ValueError:
    pass

print('\nBalance end: {}'.format(acc4.balance))


Balance start: 10
ENTER WITH: making backup of transactions for rollback
adding -50 to account
new balance would be: -40
EXIT WITH: rolling back to previous transactions
transaction resulted in ValueError (sorry cannot go in debt!)

Balance end: 10


### Conclusion

I hope you feel a little less afraid of dunder methods after reading this article. A strategic use of them makes your classes more Pythonic, because they emulate builtin types with Python-like behaviors.

As with any feature, please don’t overuse it. Operator overloading, for example, can get pretty obscure. Adding “karma” to a person object with +bob or tim << 3 is definitely possible using dunders—but might not be the most obvious or appropriate way to use these special methods. However, for common operations like comparison and additions they can be an elegant approach.

Showing each and every dunder method would make for a very long tutorial. If you want to learn more about dunder methods and the Python data model I recommend you go through the [Python reference documentation](https://docs.python.org/3/reference/datamodel.html).
